In [351]:
import numpy as np
import pandas as pd
import requests
import json
import folium
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import time
import math
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, GeoJSONDataSource, HoverTool, Slider, CustomJS
from bokeh.tile_providers import ESRI_IMAGERY, get_provider
from bokeh.layouts import column
import geopandas as gpd
from pyproj import Transformer, CRS
from shapely.ops import transform
from scipy.optimize import minimize_scalar

In [14]:
with open('variables/city_centers.json') as f_in:
    city_centers = json.load(f_in)

sf_lat = city_centers['sf'][0]
sf_lon = city_centers['sf'][1]
chicago_lat = city_centers['chicago'][0]
chicago_lon = city_centers['chicago'][1]
nyc_lat = city_centers['nyc'][0]
nyc_lon = city_centers['nyc'][1]

In [2]:
with open('json_data/sf_venues_results.json') as f_in: 
    sf_venues_results = json.load(f_in)
with open('json_data/chicago_venues_results.json') as f_in:
    chicago_venues_results = json.load(f_in)
with open('json_data/nyc_venues_results.json') as f_in:
    nyc_venues_results = json.load(f_in)

In [176]:
california = gpd.read_file('shape_data/california/california.shp')
illinois = gpd.read_file('shape_data/illinois/illinois.shp')
new_york = gpd.read_file('shape_data/new_york/new_york.shp')

In [177]:
sf_shp = california[california['NAME'] == 'San Francisco']
chicago_shp = illinois[illinois['NAME'] == 'Chicago']
nyc_shp = new_york[new_york['NAME'] == 'New York']

In [250]:
crs_wgs = CRS("EPSG:4326")
crs_merc = CRS("EPSG:3785")

wgs2merc = Transformer.from_crs(crs_wgs, crs_merc) # Usage: wgs2merc.transform(lat, lon)
merc2wgs = Transformer.from_crs(crs_merc, crs_wgs)

In [258]:
unique_ids = set()

sf_data = {
    'latitudes': [],
    'longitudes': [],
    'x_coords': [],
    'y_coords': [],
    'labels': []
}

for i, venue in enumerate(sf_venues_results):
    items_list = venue['response']['groups'][0]['items']
    for item in items_list:
        item_id = item['venue']['id']
        if item_id in unique_ids:
            continue
        else:
            unique_ids.add(item_id)
            sf_data['latitudes'].append(item['venue']['location']['lat'])
            sf_data['longitudes'].append(item['venue']['location']['lng'])
            sf_data['labels'].append(item['venue']['categories'][0]['name'])

for lat, lon in zip(sf_data['latitudes'], sf_data['longitudes']):
    x, y = wgs2merc.transform(lat, lon)
    sf_data['x_coords'].append(x)
    sf_data['y_coords'].append(y)

In [261]:
unique_ids = set()

chicago_data = {
    'latitudes': [],
    'longitudes': [],
    'x_coords': [],
    'y_coords': [],
    'labels': []
}

for i, venue in enumerate(chicago_venues_results):
    items_list = venue['response']['groups'][0]['items']
    for item in items_list:
        item_id = item['venue']['id']
        if item_id in unique_ids:
            continue
        else:
            unique_ids.add(item_id)
            chicago_data['latitudes'].append(item['venue']['location']['lat'])
            chicago_data['longitudes'].append(item['venue']['location']['lng'])
            chicago_data['labels'].append(item['venue']['categories'][0]['name'])

for lat, lon in zip(chicago_data['latitudes'], chicago_data['longitudes']):
    x, y = wgs2merc.transform(lat, lon)
    chicago_data['x_coords'].append(x)
    chicago_data['y_coords'].append(y)

In [262]:
unique_ids = set()

nyc_data = {
    'latitudes': [],
    'longitudes': [],
    'x_coords': [],
    'y_coords': [],
    'labels': []
}

for i, venue in enumerate(nyc_venues_results):
    items_list = venue['response']['groups'][0]['items']
    for item in items_list:
        item_id = item['venue']['id']
        if item_id in unique_ids:
            continue
        else:
            unique_ids.add(item_id)
            nyc_data['latitudes'].append(item['venue']['location']['lat'])
            nyc_data['longitudes'].append(item['venue']['location']['lng'])
            nyc_data['labels'].append(item['venue']['categories'][0]['name'])

for lat, lon in zip(nyc_data['latitudes'], nyc_data['longitudes']):
    x, y = wgs2merc.transform(lat, lon)
    nyc_data['x_coords'].append(x)
    nyc_data['y_coords'].append(y)

In [269]:
main_shape = sf_shp.iloc[0]['geometry'][3]

In [274]:
chicago_poly = {
    'x': [],
    'y': []
}
for point in chicago_shp.iloc[0]['geometry'].exterior.coords:
    new_coords = wgs2merc.transform(point[1], point[0])
    chicago_poly['x'].append(new_coords[0])
    chicago_poly['y'].append(new_coords[1])

In [281]:
chicago_poly['x'][0]

-9789447.263629049

In [160]:
samp_x, samp_y = latlon2mercator(37.76, -122.44)

In [173]:
tile_provider = get_provider(OSM)
sf_x, sf_y = latlon2mercator(sf_lat, sf_lon)
# p = figure(x_range = (sf_x - 100, sf_x + 100), y_range= (sf_y - 100, sf_y + 100),
#            x_axis_type = "mercator", y_axis_type = "mercator")
p = figure(x_axis_type="mercator", y_axis_type="mercator")
p.circle(x = 'x_coords', y = 'y_coords', source = sf_data)
p.annulus(x = samp_x, y = samp_y, inner_radius = 0, outer_radius = 2030.0, 
          fill_color = 'green', fill_alpha = 0.5, line_alpha = 0)
p.annulus(x = samp_x + 800, y = samp_y, inner_radius = 0, outer_radius = 2030.0, 
          fill_color = 'green', fill_alpha = 0.5, line_alpha = 0)
p.annulus(x = samp_x, y = samp_y + 800, inner_radius = 0, outer_radius = 2030.0, 
          fill_color = 'green', fill_alpha = 0.5, line_alpha = 0)
p.annulus(x = samp_x + 800, y = samp_y + 800, inner_radius = 0, outer_radius = 2030.0, 
          fill_color = 'green', fill_alpha = 0.5, line_alpha = 0)
p.add_tile(tile_provider)
show(p)

In [25]:
tile_provider = get_provider(ESRI_IMAGERY)

# range bounds supplied in web mercator coordinates
p = figure(x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)

show(p)

In [50]:
def latlon2mercator(lat, lon):
    r_earth = 6378137.000
    x = r_earth * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return (x, y)

In [52]:
def kil2mil(km):
    return km * 0.621371

In [53]:
# Returns (approximate) distance in kilometers or miles.
def calc_dist_latlon(lat_1, lon_1, lat_2, lon_2, unit = 'km'):
    r_earth = 6371 # avg radius of Earth in km
    d_lat = np.radians(lat_2 - lat_1)
    d_lon = np.radians(lon_2 - lon_1)
    
#     a = ((math.sin(d_lat/2))**2 + 
#          (math.sin(d_lon/2))**2 + 
#          (math.cos(deg2rad(lat_1)) * math.cos(deg2rad(lat_2))))
    
#     print(math.cos(deg2rad(lat_1)))
#     print(math.cos(deg2rad(lat_2)))
#     print(a)

    a = ((math.sin(d_lat/2) * math.sin(d_lat/2)) + 
         (math.cos(np.radians(lat_1)) * math.cos(np.radians(lat_2)) * 
         math.sin(d_lon/2) * math.sin(d_lon/2)))
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = r_earth * c
    
    if unit == 'km':
        return d
    elif unit == 'mi':
        return kil2mil(d)
    else:
        print("Units not recognized.")
        return 0

In [54]:
print(chicago_data['latitudes'][0])
print(chicago_data['longitudes'][0])

41.658275
-87.608822


In [83]:
lat_1 = 41.66000
lon_1 = -87.61
lat_2 = 41.67446
lon_2 = -87.61

In [84]:
calc_dist_latlon(lat_1, lon_1, lat_2, lon_2, 'mi')

0.9990891579687212

In [87]:
print(latlon2mercator(lat_1, lon_1)[1] - latlon2mercator(lat_2, lon_2)[1])

-2154.8057680875063


In [112]:
lat_1 = 41.66
lon_1 = -87.62936
lat_2 = 41.66
lon_2 = -87.61000

In [113]:
calc_dist_latlon(lat_1, lon_1, lat_2, lon_2, 'mi')

0.9993588150024879

In [114]:
print(latlon2mercator(lat_1, lon_1)[0] - latlon2mercator(lat_2, lon_2)[0])

-2155.1453417595476


In [115]:
print(sf_data['latitudes'][0])
print(sf_data['longitudes'][0])

37.71047
-122.46752


In [120]:
lat_1 = 37.71
lon_1 = -122.47
lat_2 = 37.72446
lon_2 = -122.47

In [121]:
calc_dist_latlon(lat_1, lon_1, lat_2, lon_2, 'mi')

0.9990891579682302

In [122]:
print(latlon2mercator(lat_1, lon_1)[1] - latlon2mercator(lat_2, lon_2)[1])

-2034.8916148012504


In [154]:
lat_1 = 37.71
lon_1 = -122.48830
lat_2 = 37.71
lon_2 = -122.47000

In [155]:
calc_dist_latlon(lat_1, lon_1, lat_2, lon_2, 'mi')

1.0002939531870083

In [156]:
print(latlon2mercator(lat_1, lon_1)[0] - latlon2mercator(lat_2, lon_2)[0])

-2037.1466815192252


In [245]:
# project = Transformer.from_proj(
#     pyproj.Proj(init='epsg:4326'), # source coordinate system
#     pyproj.Proj(init='epsg:3785')) # destination coordinate system

crs_source = CRS("EPSG:4326")
crs_dest = CRS("EPSG:3785")

transformer = Transformer.from_crs(crs_source, crs_dest)

In [247]:
sf_polygon = sf_shp.iloc[0]['geometry']
single_polygon = sf_polygon[3]
len(single_polygon.interiors)
for x in single_polygon.exterior.coords:
    print(transformer.transform(x[1], x[0]))
    break

(-13638249.862361172, 4548510.400085907)


In [ ]:
sf_geosource = GeoJSONDataSource(geojson = sf_shp.to_json())
chicago_geosource = GeoJSONDataSource(geojson = chicago_shp.to_json())
nyc_geosource = GeoJSONDataSource(geojson = nyc_shp.to_json())

In [259]:
p = figure(x_axis_type = "mercator", y_axis_type = "mercator")
# p.patches('xs', 'ys', source = sf_geosource, alpha = 0.5, color = 'green')
p.circle('x_coords', 'y_coords', source = sf_data)

tile_provider = get_provider(OSM)
p.add_tile(tile_provider)

show(p)

In [427]:
start_lat = chicago_data['latitudes'][0]
start_lon = chicago_data['longitudes'][0]
# new_lat = 41.6672682160591
new_lat = 41.66726835474764
new_lon = start_lon
calc_dist_latlon(start_lat, start_lon, new_lat, new_lon)

1.0000154214520725

In [425]:
def tmp_func(x):
    return abs(calc_dist_latlon(start_lat, start_lon, x, start_lon) - 1)

In [417]:
print(tmp_func(41.658275304938876))
print(tmp_func(41.6672682160591))

0.9999660923444551
1.0130674077402091e-11


In [428]:
chicago_min = minimize_scalar(tmp_func, method = 'bounded', bounds = (start_lat, start_lat + 10))

In [470]:
def find_a_mile(start_lat, start_lon, var):
    if var == 'lat':
        func_to_opt = lambda x: abs(calc_dist_latlon(start_lat, start_lon, x, start_lon, 'mi') - 1)
        bounds = (start_lat, start_lat + 10)
    elif var == 'lon':
        func_to_opt = lambda x: abs(calc_dist_latlon(start_lat, start_lon, start_lat, x, 'mi') - 1)
        bounds = (start_lon, start_lon + 10)
    else:
        print('Specify which variable to optimize')
        return
    
    optimized = minimize_scalar(func_to_opt, method = 'bounded', bounds = bounds)
    
    if var == 'lat':
        new_lat = optimized.x
        new_lon = start_lon
        axis = 1
    else:
        new_lat = start_lat
        new_lon = optimized.x
        axis = 0
    
    difference = latlon2mercator(start_lat, start_lon)[axis] - latlon2mercator(new_lat, new_lon)[axis]
    return int(abs(difference))

In [471]:
dist_1 = find_a_mile(chicago_data['latitudes'][0], chicago_data['longitudes'][0], 'lat')
dist_2 = find_a_mile(chicago_data['latitudes'][0], chicago_data['longitudes'][0], 'lon')
dist_3 = find_a_mile(chicago_data['latitudes'][-1], chicago_data['longitudes'][-1], 'lat')
dist_4 = find_a_mile(chicago_data['latitudes'][-1], chicago_data['longitudes'][-1], 'lon')

avg_dist_chicago = (dist_1 + dist_2 + dist_3 + dist_4) / 4

In [472]:
dist_1 = find_a_mile(sf_data['latitudes'][0], sf_data['longitudes'][0], 'lat')
dist_2 = find_a_mile(sf_data['latitudes'][0], sf_data['longitudes'][0], 'lon')
dist_3 = find_a_mile(sf_data['latitudes'][-1], sf_data['longitudes'][-1], 'lat')
dist_4 = find_a_mile(sf_data['latitudes'][-1], sf_data['longitudes'][-1], 'lon')

# print(dist_1, dist_2, dist_3, dist_4)

avg_dist_sf = (dist_1 + dist_2 + dist_3 + dist_4) / 4

In [473]:
dist_1 = find_a_mile(nyc_data['latitudes'][0], nyc_data['longitudes'][0], 'lat')
dist_2 = find_a_mile(nyc_data['latitudes'][0], nyc_data['longitudes'][0], 'lon')
dist_3 = find_a_mile(nyc_data['latitudes'][-1], nyc_data['longitudes'][-1], 'lat')
dist_4 = find_a_mile(nyc_data['latitudes'][-1], nyc_data['longitudes'][-1], 'lon')

# print(dist_1, dist_2, dist_3, dist_4)

avg_dist_nyc = (dist_1 + dist_2 + dist_3 + dist_4) / 4

In [474]:
print(avg_dist_chicago)

2161.5


In [477]:
p = figure(x_axis_type = "mercator", y_axis_type = "mercator")
# p.patches('xs', 'ys', source = nyc_geosource, alpha = 0.5, color = 'green')
p.circle('x_coords', 'y_coords', source = sf_data)

tile_provider = get_provider(OSM)
p.add_tile(tile_provider)

p_annulus = p.annulus(x = 'x_coords', y = 'y_coords', source = sf_data, 
                      inner_radius = 0, outer_radius = avg_dist_sf, 
                      fill_color = 'gray', fill_alpha = 0.5, line_alpha = 0)

slider = Slider(start = 0, end = 1, step = 0.01, value = 0.5)
slider.js_on_change('value',
    CustomJS(args=dict(other = p_annulus.glyph, factor = avg_dist_sf),
             code="other.outer_radius = factor * this.value"))


show(column(p, slider))

In [475]:
p = figure(x_axis_type = "mercator", y_axis_type = "mercator")
p.patch('x', 'y', source = chicago_poly, alpha = 0.5, color = 'green')
p.circle('x_coords', 'y_coords', source = chicago_data)

tile_provider = get_provider(OSM)
p.add_tile(tile_provider)

p_annulus = p.annulus(x = 'x_coords', y = 'y_coords', source = chicago_data, 
                      inner_radius = 0, outer_radius = avg_dist_chicago, 
                      fill_color = 'gray', fill_alpha = 0.5, line_alpha = 0)

slider = Slider(start = 0, end = 1, step = 0.01, value = 0.5)
# slider.js_link('value', p_annulus.glyph, 'outer_radius')
slider.js_on_change('value',
    CustomJS(args=dict(other = p_annulus.glyph, factor = avg_dist_chicago),
             code="other.outer_radius = factor * this.value"))

# p.add_tools(HoverTool())

# show(p)

show(column(p, slider))

In [476]:
p = figure(x_axis_type = "mercator", y_axis_type = "mercator")
# p.patches('xs', 'ys', source = nyc_geosource, alpha = 0.5, color = 'green')
p.circle('x_coords', 'y_coords', source = nyc_data)

tile_provider = get_provider(OSM)
p.add_tile(tile_provider)

p_annulus = p.annulus(x = 'x_coords', y = 'y_coords', source = nyc_data, 
                      inner_radius = 0, outer_radius = avg_dist_nyc, 
                      fill_color = 'gray', fill_alpha = 0.5, line_alpha = 0)

slider = Slider(start = 0, end = 1, step = 0.01, value = 0.5)
slider.js_on_change('value',
    CustomJS(args=dict(other = p_annulus.glyph, factor = avg_dist_nyc),
             code="other.outer_radius = factor * this.value"))


show(column(p, slider))

In [307]:
def gen_circle(radius, center, num_points = 75):
    data = {
        'x': [],
        'y': []
    }
    for i in range(num_points):
        angle = 2 * math.pi * i / num_points
        data['x'].append(center[0] + (radius * math.cos(angle)))
        data['y'].append(center[1] + (radius * math.sin(angle)))
        
    return data

In [315]:
square_data = {
    'x': [-2, 7, 7, -2],
    'y': [5, 5, -6, -6]
}

circle_1_data = gen_circle(2, (1, 2))
circle_2_data = gen_circle(2, (4.5, 0))
circle_3_data = gen_circle(2, (3, 2))

p = figure()

# p.patch('x', 'y', source = circle_1_data, color = 'green')
# p.patch('x', 'y', source = circle_2_data, color = 'green')
# p.patch('x', 'y', source = square_data, color = 'blue', alpha = 0.5)

p.multi_polygons(xs = [[[ square_data['x'], circle_1_data['x'], circle_2_data['x'], circle_3_data['x'] ]]],
                 ys = [[[ square_data['y'], circle_1_data['y'], circle_2_data['y'], circle_3_data['y'] ]]])

show(p)

In [324]:
square_data = {
    'x': [-5, 8, 8, -5],
    'y': [4, 4, -9, -9]
}

circle_1_data = gen_circle(3, (-0.5, -2))
circle_2_data = gen_circle(3, (3.5, -2))

circle_2_cut = {
    'x': [],
    'y': []
}

for x, y in zip(circle_2_data['x'], circle_2_data['y']):
    if x < 1.5:
        pass
    else:
        circle_2_cut['x'].append(x)
        circle_2_cut['y'].append(y)

p = figure()

p.multi_polygons(xs = [[[ square_data['x'], circle_1_data['x'], circle_2_cut['x'] ]]],
                 ys = [[[ square_data['y'], circle_1_data['y'], circle_2_cut['y'] ]]])

show(p)